In [1]:
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

#'host = 127.0.0.1 user = postgres dbname = test port = 5432 password = admin sslmode = allow'
def get_postgres_conn(conn_info):
    conn_string = f'host = {conn_info['host']} user = postgres dbname = {conn_info['dbname']} port = {conn_info['port']} password = {conn_info['password']} sslmode = allow'
    conn = psycopg2.connect(conn_string)
    return conn


def create_invoice_books(conn):
    cur = conn.cursor()
    # invoice_books_python如果存在，則刪除
    drop_sql = '''DROP TABLE IF EXISTS invoice_books_python'''
    cur.execute(drop_sql)
    conn.commit()
    
    sql = '''CREATE TABLE invoice_books_python (
            id SERIAL PRIMARY KEY,
            track VARCHAR(2) NOT NULL,
            begin_number BIGINT NOT NULL,
            end_number BIGINT NOT NULL,
            year INT NOT NULL,
            month INT NOT NULL,
            created_at TIMESTAMP NOT NULL,
            update_at TIMESTAMP NOT NULL
            )'''
    try:
        cur.execute(sql)
        conn.commit()
        print("invoice_books_pythoncreated successfully")

    except Exception as e:
        conn.rollback()
        print(f"Error occurred: {e}")

    finally:
        cur.close()

def insert_invoice_book(conn, invoice_data):
    cur = conn.cursor()
    sql = ''' INSERT INTO invoice_books_python (track, begin_number, end_number, year, month, created_at, update_at)
              VALUES (%s, %s, %s, %s, %s, %s, %s)
           '''
    try:
        if len(invoice_data) == 1:
        # 使用 execute 插入單筆資料
            cur.execute(sql, invoice_data)
        else:
            cur.executemany(sql, invoice_data)

        conn.commit()
        print("Data inserted successfully")
    
    except Exception as e:
        conn.rollback()
        print(f"Error occurred: {e}")
    
    finally:
        cur.close()
        

def create_invoice_config(conn):
    cur = conn.cursor()
    # invoices_config_python如果存在，則刪除
    drop_sql = '''DROP TABLE IF EXISTS invoices_config_python'''
    cur.execute(drop_sql)
    conn.commit()
    
    sql = '''CREATE TABLE invoices_config_python (
            id SERIAL PRIMARY KEY,  
            invoice_number VARCHAR(50) NOT NULL,
            invoice_date DATE NOT NULL,
            created_at TIMESTAMP NOT NULL,
            update_at TIMESTAMP NOT NULL,
            remark BOOLEAN NOT NULL DEFAULT true 
            )'''
    cur = conn.cursor()
    try:
        cur.execute(sql)
        conn.commit()
        print("invoices_config_python created successfully")

    except Exception as e:
        conn.rollback()
        print(f"Error occurred: {e}")
    finally:
        cur.close()
        
        
        
def insert_invoice_config(conn):
    cur = conn.cursor()
    sql ='''WITH RECURSIVE invoice_series AS (
            SELECT
                b.track,
                b.begin_number AS invoice_number,
                b.end_number,
                '2024-03-01'::date AS invoice_date_start,
                '2024-03-31'::date AS invoice_date_end,
                '2024-03-01 00:00:00'::timestamp AS created_at_start,
                '2024-03-31 23:59:59'::timestamp AS created_at_end
            FROM invoice_books_python b
            UNION ALL
            SELECT
                s.track,
                s.invoice_number + 1,
                s.end_number,
                s.invoice_date_start,
                s.invoice_date_end,
                s.created_at_start,
                s.created_at_end
            FROM invoice_series s
            WHERE s.invoice_number < s.end_number
            )
            INSERT INTO invoices_config_python (invoice_number, invoice_date, created_at, update_at, remark)
            SELECT 
                s.track || '-' || s.invoice_number::text AS invoice_number,
                -- 隨機選擇一個日期範圍內的日期
                (s.invoice_date_start + (random() * (s.invoice_date_end - s.invoice_date_start))::int)::date AS invoice_date,
                -- 隨機生成 created_at 和 update_at
                (s.created_at_start + (random() * (s.created_at_end - s.created_at_start))) AS created_at,
                (s.created_at_start + (random() * (s.created_at_end - s.created_at_start))) AS update_at,
                -- 根據條件設定 remark 欄位
                CASE 
                    WHEN s.track = 'AC' AND s.invoice_number >= 45678988 THEN False
                    ELSE True
                END AS remark
            FROM invoice_series s '''
    try:
        # 執行 CREATE TABLE 語句
        cur.execute(sql)
        # 提交變更
        conn.commit()
        print("invoices_config_python created successfully")

    except Exception as e:
        # 如果發生錯誤，回滾變更
        conn.rollback()
        print(f"Error occurred: {e}")

    finally:
        # 關閉 cursor 
        cur.close()

def create_test_info(conn,size = 10):
    #size : 可調整隨機刪除筆數，預設為刪除10筆
    cur = conn.cursor()
    # invoices_test_python如果存在，則刪除
    drop_sql = '''DROP TABLE IF EXISTS invoices_test_python'''
    cur.execute(drop_sql)
    conn.commit()
    
    # 以invoices_config_python建立invoices_test_python
    create_test_data_table = '''CREATE TABLE invoices_test_python AS
                                SELECT * FROM invoices_config_python '''
    cur.execute(create_test_data_table)
    conn.commit()
    
    # 隨機刪除幾筆不為空白發票資料 => 測資
    ramdon_delete =f'''DELETE FROM invoices_test_python
                      WHERE remark IS NOT false
                      AND id IN (
                            SELECT id
                            FROM invoices_test_python
                            WHERE remark IS NOT false
                            ORDER BY random()
                            LIMIT {size} 
                            )'''
    cur.execute(ramdon_delete)
    conn.commit()
    cur.close()
    print('測資設定完成')


def get_result_info(conn):
    sql = '''SELECT 
                config.id, 
                config.invoice_number, 
                b.track, 
                b.year, 
                b.month, 
                b.begin_number, 
                b.end_number
            FROM invoices_config_python config
            LEFT JOIN invoice_books_python b ON config.invoice_number LIKE b.track || '%'
            WHERE config.invoice_number NOT IN (
                SELECT invoice_number
                FROM invoices_test_python 
            )
            ORDER BY config.id'''
    df = pd.read_sql(sql,conn)
    return df


if __name__ == '__main__':

    # 建立連線
    conn_info = {'host':'127.0.0.1','dbname':'test', 'port':5432, 'password':'admin'}
    conn = get_postgres_conn(conn_info)
    create_invoice_books(conn)
    invoice_data = [
    ('AA', 12345600, 12345649, 113, 3, '2024-03-01 00:00:00', '2024-03-10 12:00:00'),
    ('AB', 98765400, 98765449, 113, 3, '2024-03-01 00:00:00', '2024-03-15 12:00:00'),
    ('AC', 45678900, 45678999, 113, 3, '2024-03-01 00:00:00', '2024-03-20 12:00:00')]
    insert_invoice_book(conn, invoice_data)
    create_invoice_config(conn)
    insert_invoice_config(conn)
    create_test_info(conn,size = 10)
    df = get_result_info(conn)
    conn.close()

invoice_books_pythoncreated successfully
Data inserted successfully
invoices_config_python created successfully
invoices_config_python created successfully
測資設定完成


In [2]:
df

,id,invoice_number,track,year,month,begin_number,end_number
0,25,AA-12345608,AA,113,3,12345600,12345649
1,26,AB-98765408,AB,113,3,98765400,98765449
2,44,AB-98765414,AB,113,3,98765400,98765449
3,46,AA-12345615,AA,113,3,12345600,12345649
4,111,AC-45678936,AC,113,3,45678900,45678999
5,138,AC-45678945,AC,113,3,45678900,45678999
6,141,AC-45678946,AC,113,3,45678900,45678999
7,146,AB-98765448,AB,113,3,98765400,98765449
8,182,AC-45678981,AC,113,3,45678900,45678999
9,187,AC-45678986,AC,113,3,45678900,45678999
